In [1]:
import numpy as np

In [42]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from PIL import Image
import os

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size

        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)

        combined_conv = self.conv(combined)

        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first=True, bias=True):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))
        self.fc = nn.Linear(self.hidden_dim[-1], 1)

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, seq_len, _, h, w = input_tensor.size()

        if hidden_state is not None:
            raise NotImplementedError()
        else:
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)

        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                print (h.shape)
                fc_output = self.fc(h)
                output_inner.append(fc_output)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.batch_first:
            # (b, t, c, h, w) -> (t, b, c, h, w)
            layer_output_list = [x.permute(1, 0, 2, 3, 4) for x in layer_output_list]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [43]:
X_train=np.load("/home/kumarv/renga016/Public/sst_dataset_omri/sstday_train.npy")
X_test=np.load("/home/kumarv/renga016/Public/sst_dataset_omri/sstday_valid.npy")
t, m, n = X_test.shape
sst_nanflag = np.full((m, n), True)
np.save("sst/sst_sst_omri_nanflag.npy",sst_nanflag)
print (X_train.shape, X_test.shape)

(1095, 70, 150) (1242, 70, 150)


In [44]:
X_train = X_train.reshape( X_train.shape[0],1 X_train.shape[1], X_train.shape[2])
X_train = torch.from_numpy(X_train).float().contiguous()

trainDat = []
start = 0
for i in np.arange(14,-1, -1):
    if i == 0:
        trainDat.append(X_train[start:])
    else:
        trainDat.append(X_train[start:-i])
    start += 1

train_data = torch.utils.data.TensorDataset(*trainDat)
# train_data = torch.tensor(trainDat)
del(trainDat)

train_loader = DataLoader(dataset = train_data,batch_size = 64,shuffle = True)

SyntaxError: invalid syntax (<ipython-input-44-08fefb6e459c>, line 1)

In [45]:
num_epochs = 10


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the ConvLSTM model
model = ConvLSTM(input_dim=1,   # input dimension (e.g., 3 for RGB images)
                 hidden_dim=[64, 64],   # dimensionality of the hidden state in each layer
                 kernel_size=(3, 3),   # size of the convolving kernel
                 num_layers=2).to(device)   
# Define the loss function and the optimizer
criterion = nn.MSELoss()  # or another appropriate loss function
optimizer = Adam(model.parameters(), lr=0.001)  # adjust learning rate as needed


# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, data_list in enumerate(train_loader):
        print (data_list[0].shape)
        

        # Forward pass
        outputs = model(data_list[0].unsqueeze(1).to(device))
        print (outputs[0][0].shape)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item()}')

#     # Validation loop
#     model.eval()
#     with torch.no_grad():
#         total = 0
#         correct = 0
#         for images, labels in val_loader:
#             images = images.to(device)
#             labels = labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
        
#         print(f'Validation Accuracy of the model on the test images: {100 * correct / total}%')


torch.Size([64, 1, 70, 150])
torch.Size([64, 64, 70, 150])


RuntimeError: mat1 dim 1 must match mat2 dim 0